<a href="https://colab.research.google.com/github/tollinx/VLA/blob/main/CLIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision pillow transformers open_clip_torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.0 MB/s eta 0:00:00


In [ ]:
import torch
import open_clip
from PIL import Image

# Load pretrained CLIP model + transforms
model, _, preprocess = open_clip.create_model_and_transforms(
    'ViT-B-32', pretrained='laion2b_s34b_b79k'
)
tokenizer = open_clip.get_tokenizer('ViT-B-32')


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


open_clip_model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [ ]:
from google.colab import files
uploaded = files.upload()

image_path = list(uploaded.keys())[0]
print("Using image:", image_path)

Saving algaeBloom.jpg to algaeBloom.jpg
Using image: algaeBloom.jpg


In [ ]:
from PIL import Image
import torch
from torchvision import transforms

# Define preprocessing for CLIP ViT-B/32
def preprocess_rgb(image_path, image_size=224):
    transform = transforms.Compose([
        transforms.Resize(image_size, interpolation=Image.BICUBIC),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.48145466, 0.4578275, 0.40821073],  # CLIP's RGB mean
            std=[0.26862954, 0.26130258, 0.27577711]   # CLIP's RGB std
        )
    ])

    img = Image.open(image_path).convert("RGB")
    return transform(img).unsqueeze(0)   # add batch dimension

# Preprocess the uploaded image
image = preprocess_rgb(image_path)   # ✅ use preprocess_rgb()

# Candidate text prompts
prompts = ["a flooded area", "a normal dry area"]
text = tokenizer(prompts)

# Run through CLIP
with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)

    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    # Similarity (softmax as probabilities)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)

for label, score in zip(prompts, similarity[0]):
    print(f"{label}: {score:.4f}")


a flooded area: 0.0338
a normal dry area: 0.9662
